# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

* Litesh

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# filepath 
file = "../output/cities.csv"

# read the csv file into dataframe
city_weather_df = pd.read_csv(file)

#display df
city_weather_df


,City,City.1,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,languyan,5.2618,120.0807,86.18,64,98,2.19,PH,1630654068
1,1,kamaishi,39.2667,141.8833,76.15,70,100,6.40,JP,1630654069
2,2,guozhen,34.3659,107.3590,73.33,94,100,6.04,CN,1630654070
3,3,vila velha,-20.3297,-40.2925,66.15,94,75,2.30,BR,1630654070
4,4,saint-pierre,-21.3393,55.4781,76.68,59,3,29.93,RE,1630654071
...,...,...,...,...,...,...,...,...,...,...
573,573,sao miguel do araguaia,-13.2750,-50.1628,79.79,35,88,4.09,BR,1630654486
574,574,shihezi,44.3000,86.0333,85.93,18,8,12.86,CN,1630654487
575,575,smirnykh,49.7500,142.8333,59.59,92,87,6.35,RU,1630654488
576,576,sioux lookout,50.1001,-91.9170,61.18,82,90,9.22,CA,1630654488


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Get locations and humidity
locations = city_weather_df[["Lat", "Lng"]]
weights = city_weather_df["Humidity"]

# Format heatmap
layout = {
    'width': "100%",
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',

}
#Formattting
fig = gmaps.figure(layout=layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100,
                                 point_radius = 1)
#Add the heatmap layer
fig.add_layer(heatmap_layer)

#display heatmap
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Dataframe to hold ideal weather conditions
ideal_cities = city_weather_df.loc[(city_weather_df["Max Temp"] > 70) & 
                               (city_weather_df["Max Temp"] < 80) & 
                               (city_weather_df["Wind Speed"] < 10) & 
                               (city_weather_df["Cloudiness"] == 0) &
                               #drop NA
                               (city_weather_df["Humidity"] < 75), :].reset_index(drop=True).dropna()
ideal_cities

,City,City.1,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,54,dawson,31.7735,-84.4466,74.89,74,0,7.56,US,1630654105
1,187,fiumicino,41.7675,12.2291,76.08,60,0,4.61,IT,1630653944
2,228,middelburg,-25.7751,29.4648,74.48,20,0,7.47,ZA,1630654235
3,269,avera,33.1940,-82.5271,71.26,63,0,6.64,US,1630654266
4,281,nelson bay,-32.7167,152.1500,70.29,74,0,3.00,AU,1630654275
5,447,tiznit,29.5833,-9.5000,74.82,43,0,1.05,MA,1630654397
6,463,troitskoye,52.9821,84.6757,72.81,27,0,8.79,RU,1630654409


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Dataframe hotel_df 
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng"]].copy()

# --- Add a "Hotel Name" column to the DataFrame with null values ---


hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,54,US,31.7735,-84.4466,NaN
1,187,IT,41.7675,12.2291,NaN
2,228,ZA,-25.7751,29.4648,NaN
3,269,US,33.1940,-82.5271,NaN
4,281,AU,-32.7167,152.1500,NaN
5,447,MA,29.5833,-9.5000,NaN
6,463,RU,52.9821,84.6757,NaN


In [7]:
ideal_cities

,City,City.1,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,54,dawson,31.7735,-84.4466,74.89,74,0,7.56,US,1630654105
1,187,fiumicino,41.7675,12.2291,76.08,60,0,4.61,IT,1630653944
2,228,middelburg,-25.7751,29.4648,74.48,20,0,7.47,ZA,1630654235
3,269,avera,33.1940,-82.5271,71.26,63,0,6.64,US,1630654266
4,281,nelson bay,-32.7167,152.1500,70.29,74,0,3.00,AU,1630654275
5,447,tiznit,29.5833,-9.5000,74.82,43,0,1.05,MA,1630654397
6,463,troitskoye,52.9821,84.6757,72.81,27,0,8.79,RU,1630654409


In [8]:
# --- set up parameters for querying Google Places API ---
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# --- set up a parameters dictionary ---
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop to go through each city for an API call 

for index, row in hotel_df.iterrows():
    
    # coordinates and store in variables
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # API call, hotel details for each city 
    hotel_data = requests.get(base_url, params).json()
    
    # save the first hotel name for each city
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")

City '54' found! The nearest hotel is Country Hearth Inn & Suites Dawson
-------------------------------------------------------------------------
City '187' found! The nearest hotel is Hotel Isola Sacra
-------------------------------------------------------------------------
City '228' found! The nearest hotel is Rustique Boutique Hotel
-------------------------------------------------------------------------
Could not find information for city '269'... skipping.
-------------------------------------------------------------------------
City '281' found! The nearest hotel is Ramada Resort by Wyndham Shoal Bay
-------------------------------------------------------------------------
Could not find information for city '447'... skipping.
-------------------------------------------------------------------------
Could not find information for city '463'... skipping.
-------------------------------------------------------------------------


In [9]:
#dropy na values
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

#display df
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,54,US,31.7735,-84.4466,Country Hearth Inn & Suites Dawson
1,187,IT,41.7675,12.2291,Hotel Isola Sacra
2,228,ZA,-25.7751,29.4648,Rustique Boutique Hotel
3,281,AU,-32.7167,152.1500,Ramada Resort by Wyndham Shoal Bay


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='100%'))